<a href="https://colab.research.google.com/github/prateekrai14/Stock_market_projects/blob/main/growmutualfundparser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
pd.set_option('display.max_columns', None)

#### Fetch Mutual fund urls

In [ ]:
url_list=[]
for i in range(0,78):
    r=requests.get('https://groww.in/slr/v1/search/derived/scheme?available_for_investment=true&doc_type=scheme&max_aum=&page={}&plan_type=Direct&q=&size=15&sort_by=0'.format(str(i)))
    data=r.json()
    df2=pd.DataFrame(data['content'])
    df2.head(1)
    df2['url']='https://groww.in/mutual-funds/'+df2.search_id
    url_list.extend(list(df2['url']))

In [ ]:
#### Fetch Mutual Data Investment information

In [ ]:
from bs4 import BeautifulSoup
import re
import json

In [ ]:
mutual_fund_data={}
for i in range(0,len(url_list)):
    r=requests.get(url_list[i])
    html_soup_object = BeautifulSoup(r.content, 'html.parser')
    matches=re.findall(r'.props.*',r.text)
    filtered_data = matches[0][:-len('</script></body></html>')]
    holding_dict=json.loads('{'+filtered_data)
    mutual_fund_name=holding_dict['props']['pageProps']['mf']['scheme_name']
    # if(holding_dict['props']['pageProps']['mf']['stpDetails'] is not None):
    #   muf.append(holding_dict['props']['pageProps']['mf']['scheme_name'])
    #   mutual_fund_name=holding_dict['props']['pageProps']['mf']['stpDetails']['amc_name']
    # elif (holding_dict['props']['pageProps']['mf']['swpDetails'] is None) and (holding_dict['props']['pageProps']['mf']['stpDetails'] is None):
    #   mutual_fund_name=holding_dict['props']['pageProps']['mf']['meta_title']
    #   muf.append(holding_dict['props']['pageProps']['mf']['scheme_name'])
    # else :
    #   mutual_fund_name=holding_dict['props']['pageProps']['mf']['swpDetails']['amc_name']
    #   muf.append(holding_dict['props']['pageProps']['mf']['scheme_name'])
    for hol_list in holding_dict['props']['pageProps']['mf']['holdings']:
        if hol_list['company_name'] not in list(mutual_fund_data.keys()):
            mutual_fund_data[hol_list['company_name']]=[]
            mutual_fund_data[hol_list['company_name']].append([mutual_fund_name,hol_list['corpus_per']])
        else:
            mutual_fund_data[hol_list['company_name']].append([mutual_fund_name,hol_list['corpus_per']])


#### Run simple average operation on mutual fund data

In [ ]:
import statistics

In [ ]:
stats_mutual_data=[]
for key in mutual_fund_data.keys():
    #print(mutual_fund_data[key])
    average_data=[]
    for i in range(0,len(mutual_fund_data[key])):
        #print(mutual_fund_data[key][i][1])
        average_data.append(mutual_fund_data[key][i][1])
    #print(statistics.mean(average_data))

    stats_mutual_data.append([len(mutual_fund_data[key]),key,statistics.mean(average_data)])
#stats_mutual_data
stats_df=pd.DataFrame(stats_mutual_data,columns=['Number_of_mutual_fund','Company Name','Average_Contibution'])
stats_df.sort_values(['Number_of_mutual_fund','Average_Contibution'],ascending=[False, True]).head(60)

,Number_of_mutual_fund,Company Name,Average_Contibution
2,11,NTPC Ltd.,4.718562
80,9,Bharti Airtel Ltd.,2.133433
8,9,Container Corporation Of India Ltd.,2.281289
1,9,Bharat Electronics Ltd.,3.405256
70,9,Larsen & Toubro Ltd.,4.188235
159,8,Reserve Bank of India,0.851161
0,8,Hindustan Aeronautics Ltd.,3.057470
5,8,Power Grid Corporation Of India Ltd.,3.636470
75,7,Hindalco Industries Ltd.,0.976439
9,7,NHPC Ltd.,1.730889


In [ ]:
#mutual_fund_data.keys()
#pd.DataFrame(mutual_fund_data['NTPC Ltd.'])


,0,1
0,Invesco India PSU Equity Fund Direct Growth,7.940000
1,Aditya Birla Sun Life PSU Equity Fund Direct G...,7.339960
2,LIC MF Infrastructure Fund Direct Growth,1.520000
3,Nippon India Power & Infra Fund Direct Growth,5.990000
4,SBI PSU Direct Plan Growth,5.080000
5,ICICI Prudential Infrastructure Direct Growth,6.964224
6,DSP The Infrastructure Growth and Economic Ref...,4.120000
7,HDFC Infrastructure Direct Plan Growth,3.640000
8,Canara Robeco Infrastructure Direct Growth,5.640000
9,Kotak Infrastructure and Economic Reform Fund ...,2.350000
